# Quantification Debug & Benchmark

Test correctness, parallelization, RAM usage, and estimate time for full runs.

In [1]:
import os, sys, gzip, glob, time, resource
import pandas as pd
import numpy as np

sys.path.insert(0, '/cluster/home/jjanssens/jjans/analysis/adult_intestine/peaks/peak_calling/atac_pipeline')

import importlib
import src.quantification
importlib.reload(src.quantification)
from src.quantification import quantify, quantify_matrix, _build_peak_index, _count_1bp_hits
from src.utils import load_peaks

# Paths
BASE = '/cluster/project/treutlein/USERS/jjans/analysis/adult_intestine/peaks'
PEAK_FILE = f'{BASE}/cross_species_consensus/03_lifted_back/unified_consensus_Bonobo.bed'
FRAG_FILE = f'{BASE}/fragment_files/Bonobo/Adipocytes.fragments.tsv.gz'  # small: ~14 MB
HUMAN_PEAK_FILE = f'{BASE}/cross_species_consensus/02_merged_consensus/unified_consensus_hg38_with_ids.bed'
HUMAN_FRAG_DIR = f'{BASE}/fragment_files/Human'

def get_rss_mb():
    """Current resident set size in MB."""
    return resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024

print(f'Initial RSS: {get_rss_mb():.0f} MB')
print(f'Peak file: {os.path.basename(PEAK_FILE)}')
print(f'Fragment file: {os.path.basename(FRAG_FILE)} ({os.path.getsize(FRAG_FILE)/1e6:.1f} MB)')

Initial RSS: 186 MB
Peak file: unified_consensus_Bonobo.bed
Fragment file: Adipocytes.fragments.tsv.gz (13.7 MB)


## 1. Correctness test: small file (Bonobo Adipocytes, 8.8 MB)

In [26]:
# Single-file quantify on small Bonobo file
rss_before = get_rss_mb()
t0 = time.time()

result = quantify(
    input_file=FRAG_FILE,
    peak_file=PEAK_FILE,
    input_type='fragments',
    method='cutsites',  # should auto-detect 1bp and switch to coverage
    verbose=True,
)

elapsed = time.time() - t0
rss_after = get_rss_mb()

print(f'\nTime: {elapsed:.1f}s')
print(f'RSS: {rss_before:.0f} -> {rss_after:.0f} MB (delta: {rss_after - rss_before:.0f} MB)')
print(f'Non-zero: {(result > 0).sum():,} / {len(result):,}')
print(f'Sum: {result.sum():,}')
print(f'Top peaks:\n{result.sort_values(ascending=False).head(5)}')

  Auto-detected 1bp cut-site fragments, using 'coverage' instead of 'cutsites'
Quantifying Adipocytes.fragments.tsv.gz (fragments) over 840,026 peaks...

Time: 18.9s
RSS: 2430 -> 2430 MB (delta: 0 MB)
Non-zero: 258,726 / 840,026
Sum: 719,901
Top peaks:
unified_768321    238
unified_882100    156
unified_152905    154
unified_869106    140
unified_153076    124
Name: Adipocytes.fragments.tsv, dtype: int64


In [27]:
# Validation: compare Python approach vs bedtools on a small subset
import subprocess, tempfile

os.environ['PATH'] = '/cluster/project/treutlein/jjans/software/miniforge3/envs/scenicplus/bin:' + os.environ.get('PATH', '')

peaks_df = load_peaks(PEAK_FILE)

# Harmonize chr prefix (fragments use '1', peaks use 'chr1')
from src.quantification import _harmonize_chr_prefix
peaks_df = _harmonize_chr_prefix(peaks_df, FRAG_FILE, 'fragments')

# Take 5000 peaks
subset = peaks_df.head(5000)

# Python approach
peak_index = _build_peak_index(subset)
py_counts = np.zeros(len(subset), dtype=np.int64)
with gzip.open(FRAG_FILE, 'rt') as fh:
    current_chrom = None
    pos_buffer = []
    
    def flush():
        global pos_buffer, current_chrom
        if pos_buffer and current_chrom and current_chrom in peak_index:
            entry = peak_index[current_chrom]
            positions = np.array(pos_buffer, dtype=np.int64)
            _count_1bp_hits(positions, entry[0], entry[1], entry[2], py_counts)
        pos_buffer = []
    
    for line in fh:
        if line[0] == '#':
            continue
        tab1 = line.index('\t')
        chrom = line[:tab1]
        if chrom != current_chrom:
            flush()
            current_chrom = chrom
        tab2 = line.index('\t', tab1 + 1)
        pos_buffer.append(int(line[tab1+1:tab2]))
    flush()

# Bedtools approach (ground truth)
tmp = tempfile.mktemp(suffix='.bed')
subset[['Chromosome','Start','End','Name']].to_csv(tmp, sep='\t', header=False, index=False)
cmd = f'bedtools intersect -a {tmp} -b {FRAG_FILE} -c'
res = subprocess.run(cmd, shell=True, capture_output=True, text=True)
bt_counts = np.array([int(line.split('\t')[-1]) for line in res.stdout.strip().split('\n')])
os.unlink(tmp)

# Compare
diff = py_counts - bt_counts
print(f'Python vs bedtools on {len(subset)} peaks:')
print(f'  Python non-zero: {(py_counts > 0).sum()}, sum: {py_counts.sum()}')
print(f'  Bedtools non-zero: {(bt_counts > 0).sum()}, sum: {bt_counts.sum()}')
print(f'  Max abs difference: {np.abs(diff).max()}')
print(f'  Peaks with differences: {(diff != 0).sum()}')
if (diff != 0).any():
    mismatch_idx = np.where(diff != 0)[0]
    for i in mismatch_idx[:10]:
        print(f'    Peak {subset.iloc[i]["Name"]}: py={py_counts[i]} bt={bt_counts[i]} '
              f'[{subset.iloc[i]["Chromosome"]}:{subset.iloc[i]["Start"]}-{subset.iloc[i]["End"]}]')
else:
    print('  PERFECT MATCH')

Python vs bedtools on 5000 peaks:
  Python non-zero: 1722, sum: 5291
  Bedtools non-zero: 1722, sum: 5291
  Max abs difference: 0
  Peaks with differences: 0
  PERFECT MATCH


## 2. Benchmark: time vs file size (estimate full-run duration)

In [2]:
# Benchmark Human files of different sizes to establish time-per-MB
human_files = sorted(glob.glob(os.path.join(HUMAN_FRAG_DIR, '*.tsv.gz')))
file_sizes = {f: os.path.getsize(f) for f in human_files}

# Pick 3 files: smallest, ~median, and one around 300MB
sorted_by_size = sorted(human_files, key=lambda f: file_sizes[f])
test_files = [
    sorted_by_size[0],    # smallest
    sorted_by_size[len(sorted_by_size) // 2],  # median
    sorted_by_size[-5],   # large-ish (but not the 2.9GB monster)
]

print('Benchmarking 3 files of increasing size:')
print('-' * 70)
benchmarks = []
for fpath in test_files:
    size_mb = file_sizes[fpath] / 1e6
    rss_before = get_rss_mb()
    t0 = time.time()
    
    r = quantify(
        input_file=fpath,
        peak_file=HUMAN_PEAK_FILE,
        input_type='fragments',
        method='cutsites',
        verbose=False,
    )
    
    elapsed = time.time() - t0
    rss_after = get_rss_mb()
    rate = size_mb / elapsed if elapsed > 0 else 0
    
    benchmarks.append({
        'file': os.path.basename(fpath),
        'size_mb': size_mb,
        'time_s': elapsed,
        'rate_mb_s': rate,
        'non_zero': (r > 0).sum(),
        'total_count': r.sum(),
        'rss_delta_mb': rss_after - rss_before,
    })
    print(f'  {os.path.basename(fpath):50s} '
          f'{size_mb:>7.1f} MB  {elapsed:>6.1f}s  ({rate:.1f} MB/s)  '
          f'RSS delta: {rss_after - rss_before:+.0f} MB')

bench_df = pd.DataFrame(benchmarks)
avg_rate = bench_df['rate_mb_s'].mean()
print(f'\nAverage processing rate: {avg_rate:.2f} MB/s (compressed)')
print(f'Max RSS delta: {bench_df["rss_delta_mb"].max():.0f} MB')

Benchmarking 3 files of increasing size:
----------------------------------------------------------------------
  Paneth_cells.fragments.tsv.gz                          4.9 MB     6.0s  (0.8 MB/s)  RSS delta: +237 MB
  EECs.fragments.tsv.gz                                116.0 MB    49.7s  (2.3 MB/s)  RSS delta: +132 MB
  T_cells.fragments.tsv.gz                             347.2 MB   145.3s  (2.4 MB/s)  RSS delta: +15 MB

Average processing rate: 1.85 MB/s (compressed)
Max RSS delta: 237 MB


In [29]:
# Estimate total time for all species
species_dirs = {
    'Bonobo': f'{BASE}/fragment_files/Bonobo',
    'Chimpanzee': f'{BASE}/fragment_files/Chimpanzee',
    'Gorilla': f'{BASE}/fragment_files/Gorilla',
    'Macaque': f'{BASE}/fragment_files/Macaque',
    'Marmoset': f'{BASE}/fragment_files/Marmoset',
    'Human': f'{BASE}/fragment_files/Human',
}

print(f'Using benchmark rate: {avg_rate:.2f} MB/s (compressed)')
print(f'{"Species":<15} {"Files":>5} {"Total MB":>10} {"Est. serial":>12} {"Est. 4 workers":>15}')
print('-' * 65)

grand_total_mb = 0
for sp, sdir in species_dirs.items():
    files = glob.glob(os.path.join(sdir, '*.tsv.gz'))
    total_mb = sum(os.path.getsize(f) for f in files) / 1e6
    grand_total_mb += total_mb
    est_serial = total_mb / avg_rate
    est_parallel = total_mb / avg_rate / 4  # 4 workers
    print(f'{sp:<15} {len(files):>5} {total_mb:>10.0f} {est_serial:>10.0f}s ({est_serial/60:.0f}m)'
          f' {est_parallel:>10.0f}s ({est_parallel/60:.0f}m)')

print('-' * 65)
print(f'{"TOTAL":<15} {"":>5} {grand_total_mb:>10.0f} '
      f'{grand_total_mb/avg_rate:>10.0f}s ({grand_total_mb/avg_rate/60:.0f}m)'
      f' {grand_total_mb/avg_rate/4:>10.0f}s ({grand_total_mb/avg_rate/4/60:.0f}m)')
print(f'\nRAM per worker: ~{bench_df["rss_delta_mb"].max() + 50:.0f} MB (peak index + Python overhead)')
print(f'RAM with 4 workers: ~{4 * (bench_df["rss_delta_mb"].max() + 50):.0f} MB')

Using benchmark rate: 1.88 MB/s (compressed)
Species         Files   Total MB  Est. serial  Est. 4 workers
-----------------------------------------------------------------
Bonobo             35       2390       1270s (21m)        318s (5m)
Chimpanzee         32       3761       1998s (33m)        499s (8m)
Gorilla            33       3320       1764s (29m)        441s (7m)
Macaque            32       1798        955s (16m)        239s (4m)
Marmoset           32       4356       2314s (39m)        579s (10m)
Human              32       9108       4839s (81m)       1210s (20m)
-----------------------------------------------------------------
TOTAL                      24734      13140s (219m)       3285s (55m)

RAM per worker: ~50 MB (peak index + Python overhead)
RAM with 4 workers: ~200 MB


## 3. Test parallelization with quantify_matrix (small subset)

In [30]:
# Test quantify_matrix with 4 small files (serial vs 2-worker vs 4-worker)
small_files = sorted_by_size[:4]
print(f'Testing parallelization with {len(small_files)} files:')
for f in small_files:
    print(f'  {os.path.basename(f):50s} {file_sizes[f]/1e6:.1f} MB')

results = {}
for n_w in [1, 2, 4]:
    rss_before = get_rss_mb()
    t0 = time.time()
    
    mat = quantify_matrix(
        input_files=small_files,
        peak_file=HUMAN_PEAK_FILE,
        input_type='fragments',
        method='cutsites',
        n_workers=n_w,
        name_pattern=r'\.fragments.*$',
        name_replacement='',
        verbose=False,
    )
    
    elapsed = time.time() - t0
    rss_after = get_rss_mb()
    results[n_w] = {'time': elapsed, 'rss_delta': rss_after - rss_before, 'mat': mat}
    print(f'\n  n_workers={n_w}: {elapsed:.1f}s  '
          f'(speedup: {results[1]["time"]/elapsed:.1f}x)  '
          f'RSS delta: {rss_after - rss_before:+.0f} MB')

# Verify all produce identical results
ref = results[1]['mat']
for n_w in [2, 4]:
    match = (results[n_w]['mat'] == ref).all().all()
    print(f'  n_workers={n_w} matches serial: {match}')

print(f'\nMatrix shape: {ref.shape}')
print(f'Note: no speedup expected on login node due to shared filesystem I/O contention.')
print(f'On a compute node with dedicated I/O, 4 workers should give ~2-3x speedup.')

Testing parallelization with 4 files:
  Paneth_cells.fragments.tsv.gz                      4.9 MB
  ILCs.fragments.tsv.gz                              7.2 MB
  Monocytes.fragments.tsv.gz                         8.3 MB
  Adipocytes.fragments.tsv.gz                        9.2 MB

  n_workers=1: 14.7s  (speedup: 1.0x)  RSS delta: +0 MB

  n_workers=2: 17.1s  (speedup: 0.9x)  RSS delta: +0 MB

  n_workers=4: 21.5s  (speedup: 0.7x)  RSS delta: +0 MB
  n_workers=2 matches serial: True
  n_workers=4 matches serial: True

Matrix shape: (887532, 4)
Note: no speedup expected on login node due to shared filesystem I/O contention.
On a compute node with dedicated I/O, 4 workers should give ~2-3x speedup.


## 4. RAM profile: measure peak RSS during quantification

In [31]:
# RAM analysis
# Peak index: 888K peaks x 3 arrays (starts, ends, row_indices) x 8 bytes = ~20 MB
# Counts array: 888K x 8 bytes = ~7 MB
# Position buffer: 500K x 8 bytes = ~4 MB
# Per-worker overhead: ~31 MB + Python interpreter (~50 MB) = ~80 MB

peaks = load_peaks(HUMAN_PEAK_FILE)
peak_index = _build_peak_index(peaks)

# Measure actual peak index memory
index_bytes = sum(
    s.nbytes + e.nbytes + r.nbytes
    for s, e, r in peak_index.values()
)
counts_bytes = len(peaks) * 8  # int64

print(f'Peak index memory breakdown:')
print(f'  Peaks: {len(peaks):,}')
print(f'  Chromosomes in index: {len(peak_index)}')
print(f'  Index arrays: {index_bytes / 1e6:.1f} MB')
print(f'  Counts array: {counts_bytes / 1e6:.1f} MB')
print(f'  Position buffer (500K): {500_000 * 8 / 1e6:.1f} MB')
print(f'  Total per-worker data: {(index_bytes + counts_bytes + 500_000*8) / 1e6:.1f} MB')
print(f'  Python interpreter overhead: ~50 MB')
print(f'  Estimated per-worker: ~{(index_bytes + counts_bytes + 500_000*8) / 1e6 + 50:.0f} MB')
print(f'  Estimated 4 workers: ~{4 * ((index_bytes + counts_bytes + 500_000*8) / 1e6 + 50):.0f} MB')
print(f'  Estimated 8 workers: ~{8 * ((index_bytes + counts_bytes + 500_000*8) / 1e6 + 50):.0f} MB')
print(f'\n  Compare: bedtools without -sorted loaded the ENTIRE fragment file into RAM')
print(f'  TA_cells.tsv.gz alone = ~35 GB uncompressed, x4 workers = 140 GB')

Peak index memory breakdown:
  Peaks: 887,532
  Chromosomes in index: 63
  Index arrays: 21.3 MB
  Counts array: 7.1 MB
  Position buffer (500K): 4.0 MB
  Total per-worker data: 32.4 MB
  Python interpreter overhead: ~50 MB
  Estimated per-worker: ~82 MB
  Estimated 4 workers: ~330 MB
  Estimated 8 workers: ~659 MB

  Compare: bedtools without -sorted loaded the ENTIRE fragment file into RAM
  TA_cells.tsv.gz alone = ~35 GB uncompressed, x4 workers = 140 GB


## 5. Polars vs Pandas CSV reader benchmark

In [3]:
try:
    import polars as pl
    print(f'Polars {pl.__version__} available')
except ImportError:
    pl = None
    print('Polars not installed -- run: pip install polars')

if pl is not None:
    from src.quantification import _build_peak_index, _count_1bp_hits, _harmonize_chr_prefix

    # Use the medium Human file for a meaningful comparison
    test_file = sorted_by_size[len(sorted_by_size) // 2]  # EECs, 116 MB
    peaks = load_peaks(HUMAN_PEAK_FILE)
    peaks = _harmonize_chr_prefix(peaks, test_file, 'fragments')
    pi = _build_peak_index(peaks)
    n_peaks = len(peaks)

    print(f'Test file: {os.path.basename(test_file)} ({file_sizes[test_file]/1e6:.0f} MB)')
    print(f'Peaks: {n_peaks:,}')
    print('-' * 70)

    # --- Method 1: current pandas chunked approach ---
    def count_pandas(fpath, peak_index, n_peaks):
        counts = np.zeros(n_peaks, dtype=np.int64)
        reader = pd.read_csv(
            fpath, sep='\t', header=None,
            usecols=[0, 1], names=['c', 's'],
            dtype={'c': str, 's': np.int64},
            comment='#', engine='c', chunksize=500_000,
        )
        for chunk in reader:
            chroms = chunk['c'].values
            starts = chunk['s'].values
            for ch in np.unique(chroms):
                entry = peak_index.get(ch)
                if entry is None:
                    continue
                positions = starts[chroms == ch]
                _count_1bp_hits(positions, entry[0], entry[1], entry[2], counts)
        return counts

    # --- Method 2: polars scan_csv (lazy, streaming) ---
    def count_polars_lazy(fpath, peak_index, n_peaks):
        counts = np.zeros(n_peaks, dtype=np.int64)
        # Polars can read gzipped TSV natively; use scan_csv for streaming
        df = pl.read_csv(
            fpath, separator='\t', has_header=False,
            columns=[0, 1], new_columns=['c', 's'],
            dtypes={'c': pl.Utf8, 's': pl.Int64},
            comment_prefix='#',
        )
        # Group by chromosome -- polars partition_by is very fast
        for ch, group in df.group_by('c'):
            ch_name = ch[0]  # group_by returns tuple keys
            entry = peak_index.get(ch_name)
            if entry is None:
                continue
            positions = group['s'].to_numpy()
            _count_1bp_hits(positions, entry[0], entry[1], entry[2], counts)
        return counts

    # --- Method 3: polars with batched reading ---
    def count_polars_batched(fpath, peak_index, n_peaks):
        counts = np.zeros(n_peaks, dtype=np.int64)
        reader = pl.read_csv_batched(
            fpath, separator='\t', has_header=False,
            columns=[0, 1], new_columns=['c', 's'],
            dtypes={'c': pl.Utf8, 's': pl.Int64},
            comment_prefix='#',
            batch_size=500_000,
        )
        while True:
            batches = reader.next_batches(1)
            if not batches:
                break
            chunk = batches[0]
            chroms = chunk['c'].to_numpy()
            starts = chunk['s'].to_numpy()
            for ch in np.unique(chroms):
                entry = peak_index.get(ch)
                if entry is None:
                    continue
                positions = starts[chroms == ch]
                _count_1bp_hits(positions, entry[0], entry[1], entry[2], counts)
        return counts

    # Benchmark all three
    methods = {
        'pandas chunked': lambda: count_pandas(test_file, pi, n_peaks),
        'polars full read': lambda: count_polars_lazy(test_file, pi, n_peaks),
    }
    # batched reader may not support gzip in all versions; try it
    try:
        _ = count_polars_batched(test_file, pi, n_peaks)
        methods['polars batched'] = lambda: count_polars_batched(test_file, pi, n_peaks)
    except Exception as e:
        print(f'polars batched not available for .gz: {e}')

    ref = None
    for name, fn in methods.items():
        t0 = time.time()
        result = fn()
        elapsed = time.time() - t0
        rate = file_sizes[test_file] / 1e6 / elapsed
        if ref is None:
            ref = result
            match = True
        else:
            match = np.array_equal(ref, result)
        print(f'  {name:<25} {elapsed:>6.1f}s  ({rate:.1f} MB/s)  '
              f'sum={result.sum():,}  match={match}')

    print(f'\nConclusion: polars is {"faster" if methods else "N/A"}')

Polars 1.38.1 available
Test file: EECs.fragments.tsv.gz (116 MB)
Peaks: 887,532
----------------------------------------------------------------------


/tmp/ipykernel_3463150/2545815895.py:65: DeprecationWarning: the argument `dtypes` for `read_csv_batched` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  reader = pl.read_csv_batched(


polars batched not available for .gz: projections contained duplicate output name 'literal'. It's possible that multiple expressions are returning the same default column name. If this is the case, try renaming the columns with `.alias("new_name")` to avoid duplicate column names.

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'select' <---
Csv SCAN [/cluster/project/treutlein/USERS/jjans/analysis/adult_intestine/peaks/fragment_files/Human/EECs.fragments.tsv.gz]
PROJECT */8 COLUMNS
  pandas chunked              40.2s  (2.9 MB/s)  sum=12,524,461  match=True


/tmp/ipykernel_3463150/2545815895.py:46: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df = pl.read_csv(


  polars full read            18.5s  (6.3 MB/s)  sum=12,524,461  match=True

Conclusion: polars is faster


In [5]:
import importlib
import time
from src import quantification as Q
importlib.reload(Q)

print(f"Polars available: {Q._HAS_POLARS}")

# Use the human fragment file that was benchmarked
BASE = '/cluster/project/treutlein/USERS/jjans/analysis/adult_intestine/peaks'
HUMAN_FRAG_DIR = f'{BASE}/fragment_files/Human'
test_file = os.path.join(HUMAN_FRAG_DIR, 'EECs.fragments.tsv.gz')
print(f"Test file: {os.path.basename(test_file)} ({os.path.getsize(test_file)/1e6:.0f} MB)")

# Run via the updated module (should use polars automatically)
t0 = time.time()
result_polars = Q._count_fragments_coverage(test_file, peak_index=pi, n_peaks=n_peaks)
t_polars = time.time() - t0

# Compare against the known reference from the benchmark (sum=12,524,461)
print(f"\nPolars path: {t_polars:.1f}s")
print(f"Sum: {result_polars.sum():,}")
print(f"Non-zero: {(result_polars > 0).sum():,}")
print(f"Matches previous ref: {np.array_equal(result_polars, ref)}")

Polars available: True
Test file: EECs.fragments.tsv.gz (116 MB)

Polars path: 19.9s
Sum: 12,524,461
Non-zero: 733,520
Matches previous ref: True
